In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
plt.style.use('nice.mplstyle')
import scipy.stats as st

In [ ]:
def active_integrator(record, trigger, integration_length):
    '''
    Simulate the "active integration" circuit in software.
    This isn't quite right because it's operating on discrete data points,
    but it will produce a mathematically similar shape as the actual reset circuit.
    '''
    running_sum = 0
    ret = np.zeros_like(record)

    i = 0
    while i < len(record):
        d = record[i]
        running_sum += d
        ret[i] = running_sum

        if running_sum < trigger:
            i += 1
            continue
        
        # Perform the "active reset" integration
        endpoint = min(i + integration_length, len(record))
        j = i + 1
        while j < endpoint:
            running_sum += record[j]
            ret[j] = running_sum
            j += 1
        running_sum = 0
        i = j

    return ret

In [ ]:
def exponential(t, t0, tau, amplitude):
    ret = amplitude * np.exp(-(t - t0) / tau)
    ret[t < t0] = 0
    return ret

# simulate a SiPM pulse as many microcell firings
crystal_tau = 40
num_firings = 200
# The pulse locations are exponentially distributed across time
pulse_locations = st.expon(scale=crystal_tau).rvs(num_firings)

# The SiPM tau determines the decay constant per microcell.
# It sets the total pulse duration and rise time
sipm_tau = 50
times = np.linspace(0, 1000, num=800)
pulse = np.zeros_like(times)
for loc in pulse_locations:
    pulse += exponential(
        times, loc, sipm_tau, 1
    )

# Integrate the pulse using the "circuit" equivalent function
integrated = active_integrator(pulse, 10, 5*sipm_tau)

rug_yval = np.ones_like(pulse_locations) * 0.1
fig, ax = plt.subplots()
ax.scatter(pulse_locations, rug_yval, label='microcell firing locations', color='black', s=1)
ax.plot(times, pulse / pulse.max(), label='(normalized) summation of microcell currents', color='red')
ax.plot(times, integrated / integrated.max(), label='(normalized) actively-reset integrated pulse', color='blue')
ax.set(
    xlabel='time (arbitrary)',
    ylabel='normalized amplitude',
    title='intended active integrator shape'
)
ax.legend()